<a href="https://colab.research.google.com/github/katetarasenko/Kurs4_Practical_7_FIT_4-10_Tarasenko/blob/main/Practical7_FIT_4_10_Tarasenko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357280 sha256=aa8da22a50dcb50d551e6833da066f822727d74516b738facf298ed3091819f0
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [7]:
from surprise import Dataset
from surprise import Reader
import pandas as pd

# Завантаження датасету MovieLens 100k
data = Dataset.load_builtin('ml-100k')

# Створення pandas DataFrame для виводу перших 5 рядків
df = pd.DataFrame(data.raw_ratings, columns=['user', 'item', 'rating', 'timestamp'])
print(df.head(5))

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
  user item  rating  timestamp
0  196  242     3.0  881250949
1  186  302     3.0  891717742
2   22  377     1.0  878887116
3  244   51     2.0  880606923
4  166  346     1.0  886397596


KNN and SVD algorythms

In [8]:
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate


algo_svd = SVD()

algo_knn = KNNBasic()

# Крос-валідація для SVD
cv_svd = cross_validate(algo_svd, data, measures=['MAE'], cv=5, verbose=True)
# Крос-валідація для KNN
cv_knn = cross_validate(algo_knn, data, measures=['MAE'], cv=5, verbose=True)

Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7374  0.7347  0.7394  0.7361  0.7382  0.7372  0.0016  
Fit time          2.96    2.06    1.46    1.81    2.01    2.06    0.50    
Test time         0.42    0.14    0.15    0.19    0.21    0.22    0.10    
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7716  0.7718  0.7708  0.7764  0.7726  0.7726  0.0020  
Fit time          0.38    0.42    0.40    0.40    0.36    0.39    0.02    
Test time

In [18]:
# Обчислення середнього значення MAE
mae_svd = sum(cv_svd['test_mae']) / len(cv_svd['test_mae'])
mae_knn = sum(cv_knn['test_mae']) / len(cv_knn['test_mae'])

print(f"MAE для SVD: {mae_svd}")
print(f"MAE для KNN: {mae_knn}")

# Вибір найкращого алгоритму
best_algo = algo_svd if mae_svd < mae_knn else algo_knn
print(f"Best algorythm: {'SVD' if best_algo == algo_svd else 'KNNBasic'}")


MAE для SVD: 0.7371549125059766
MAE для KNN: 0.7726441354238547
Best algorythm: SVD


In [10]:
trainset = data.build_full_trainset()
best_algo.fit(trainset)

In [11]:
# Отримання рекомендацій для користувача (наприклад, користувач з ID 50)
user_id = str(50)
items = df['item'].unique()

# Фільми, які користувач ще не оцінив
items_to_rate = [item for item in items if not trainset.knows_item(item)]

# Прогнозування рейтингів та сортування за значенням
predictions = [best_algo.predict(user_id, item) for item in items_to_rate]
recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

# Виведення топ-10 рекомендацій
print("Рекомендовані фільми для користувача 196:")
for rec in recommendations:
    print(f"Фільм ID: {rec.iid}, Прогнозований рейтинг: {rec.est}")

Рекомендовані фільми для користувача 196:
Фільм ID: 483, Прогнозований рейтинг: 4.676951948508005
Фільм ID: 187, Прогнозований рейтинг: 4.502283593722947
Фільм ID: 408, Прогнозований рейтинг: 4.478504794249189
Фільм ID: 12, Прогнозований рейтинг: 4.452236648770499
Фільм ID: 50, Прогнозований рейтинг: 4.4386071293112135
Фільм ID: 56, Прогнозований рейтинг: 4.4128058773938195
Фільм ID: 114, Прогнозований рейтинг: 4.397550483616123
Фільм ID: 45, Прогнозований рейтинг: 4.393208713577024
Фільм ID: 251, Прогнозований рейтинг: 4.392025017024007
Фільм ID: 302, Прогнозований рейтинг: 4.391888408426491


Завдання 2

In [17]:
from surprise import Dataset, Reader, SlopeOne
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load the MovieLens 100k dataset
data = Dataset.load_builtin('ml-100k')

# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

algo = SlopeOne()
algo.fit(trainset)
predictions = algo.test(testset)

rmse = accuracy.rmse(predictions)

from collections import defaultdict

def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Sort the predicted ratings for each user and get the top N
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions, n=10)


user_id = '196'
print(f"Top 10 recommendations for user {user_id}:")
for iid, rating in top_n[user_id]:
    print(f"Movie ID: {iid}, Predicted Rating: {rating:.2f}")

RMSE: 0.9493
Top 10 recommendations for user 196:
Movie ID: 306, Predicted Rating: 4.11
Movie ID: 269, Predicted Rating: 4.09
Movie ID: 116, Predicted Rating: 4.07
Movie ID: 257, Predicted Rating: 3.96
Movie ID: 13, Predicted Rating: 3.67
Movie ID: 66, Predicted Rating: 3.67
Movie ID: 287, Predicted Rating: 3.67
Movie ID: 762, Predicted Rating: 3.43
Movie ID: 108, Predicted Rating: 3.40
Movie ID: 94, Predicted Rating: 3.26


The name "Slope One" comes from the linear relationship between the ratings of items. The algorithm assumes that the difference between the ratings for any two items remains consistent across users.
RMSE measures the average difference between values predicted by a model and the actual values. the model is considered to be good at predicting when the RMSE is 0.8 or higher. in this case the RMSE is 0.9493 which is a good result and means that the model is capable of making descent predictions for a specific user/